# Natural Language Processing

## Tugas 3: Information Retrieval

### Mekanisme

Anda hanya diwajibkan untuk mengumpulkan file ini saja ke uploader yang disediakan di https://elearning.uai.ac.id/. Ganti nama file ini saat pengumpulan menjadi **tugas3_NIM.ipynb**.

**Keterlambatan**: Pengumpulan tugas yang melebihi tenggat yang telah ditentukan tidak akan diterima. Keterlambatan akan berakibat pada nilai nol untuk tugas ini.

**Kolaborasi**: Anda diperbolehkan untuk berdiskusi dengan teman Anda, tetapi dilarang keras menyalin kode maupun tulisan dari teman Anda.

### Petunjuk

Pastikan jawaban Anda singkat, padat, dan jelas. Mayoritas pertanyaan yang diberikan dapat dijawab dalam 3-4 kalimat saja.

### Catatan ⚠️

1. Anda mungkin membutuhkan akses GPU untuk mengerjakan tugas ini. Untuk memudahkan, Anda dapat menggunakan akses GPU gratis dari [Google Colab](https://colab.research.google.com/).
2. Untuk bagian 3 dari tugas ini, Anda memerlukan akses ke model [Llama-3.2-1B](https://huggingface.co/meta-llama/Llama-3.2-1B).

In [61]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [62]:
# Jalani cell ini untuk menginstalasi dependencies untuk memuat data dan menjalankan model
!pip install huggingface_hub fastparquet pyarrow rank_bm25

## 1. Spare Matrix Retrieval (15 poin)

*Acknowledgement:* Data yang digunakan dalam tugas ini adalah [MS MARCO](https://huggingface.co/datasets/microsoft/ms_marco), sebuah koleksi dataset yang digunakan untuk mengembangkan model deep learning dalam kasus pencarian. Kumpulan data ini adalah jawaban pertanyaan berdasarkan 100.000 pertanyaan Bing asli dan jawaban yang dibuat manusia.

Untuk bagian ini, gunakan `df_train` untuk menjawab soal-soal yang diberikan.

In [81]:
splits = {
    "validation": "v1.1/validation-00000-of-00001.parquet",
    "train": "v1.1/train-00000-of-00001.parquet",
    "test": "v1.1/test-00000-of-00001.parquet",
}
df_train = pd.read_parquet("hf://datasets/microsoft/ms_marco/" + splits["train"])
df_val = pd.read_parquet("hf://datasets/microsoft/ms_marco/" + splits["validation"])
df_test = pd.read_parquet("hf://datasets/microsoft/ms_marco/" + splits["test"])

In [84]:
df_train.tail(10)


,answers,passages,query,query_id,query_type,wellFormedAnswers
82316,[Lactobacillus and Yogurt.],"{'is_selected': [0, 0, 0, 0, 1, 0, 1, 0, 0], '...",different types names of good bacteria in food,102119,entity,[]
82317,[Sarmations],"{'is_selected': [0, 0, 0, 0, 0, 0, 1], 'passag...",who invented the saddle for horses,102120,person,[]
82318,"[Arepa flour, cheese, and 1/8 teaspoon salt.]","{'is_selected': [1, 0, 0, 0, 0, 0, 0], 'passag...",what are the main ingredients for arepas,102121,entity,[]
82319,[Oak Tree],"{'is_selected': [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]...",meaning of the name ayla,102122,description,[]
82320,[$6-$16 a square foot.],"{'is_selected': [0, 0, 0, 0, 1, 0], 'passage_t...",hydronic radiant floor heating cost per square...,102123,numeric,[]
82321,[The act or action of propagating as a increas...,"{'is_selected': [1, 0, 0], 'passage_text': ['d...",meaning of propagation,102124,description,[]
82322,[Yes],"{'is_selected': [0, 0, 1, 0, 0, 0, 0, 0, 0], '...",do you have to do a phd to be a clinical psych...,102125,description,[]
82323,[Chablis],"{'is_selected': [0, 1, 0, 0, 0, 0], 'passage_t...",what wine goes with oysters,102126,entity,[]
82324,[1 Lithium carbonate 150 mg capsules. Lithium ...,"{'is_selected': [0, 0, 0, 1, 0, 0, 0, 0, 0], '...",what strengths does lithium come in,102127,description,[]
82325,[Burdick & Jackson solvents are arranged in or...,"{'is_selected': [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]...",what is polarity index definition,102128,description,[]


### Soal 1.1 (2 poin)

Terdapat beberapa baris yang tidak memiliki passage yang relevan, i.e. semua elemen `is_selected` bernilai nol. Hapus semua baris tersebut.

In [65]:
df_train = df_train[df_train['passages'].apply(lambda x: any(x.get('is_selected', [])))]
df_val = df_val[df_val['passages'].apply(lambda x: any(x.get('is_selected', [])))]


assert df_train.shape[0] == 79704
assert df_val.shape[0] == 9706

In [66]:
print(df_train.shape)
print(df_val.shape)

(79704, 6)
(9706, 6)


### Soal 1.2 (2 poin)

Lengkapi fungsi untuk menghitung average precision di bawah ini. Nilai parameter `y_true` adalah `is_selected` dari kolom `passages`, sedangkan `y_scores` adalah nilai yang digunakan untuk mengurutkan passage berdasarkan relevansi.

In [67]:
def average_precision(y_true, y_scores):
    y_true = y_true[np.argsort(y_scores)[::-1]]
    precisions = [np.sum(y_true[:i+1]) / (i+1) for i in range(len(y_true)) if y_true[i]]
    return np.mean(precisions) if precisions else 0.0


assert np.isclose(average_precision(np.array([1, 0, 1]), np.array([0.9, 0.8, 0.7])), 0.8333, atol=1e-4)
assert np.isclose(average_precision(np.array([1, 0, 1]), np.array([0.95, 0.8, 0.9])), 1.0)
assert np.isclose(average_precision(np.array([1, 0, 0]), np.array([0.0, 0.8, 0.9])), 1 / 3)

In [68]:
print(average_precision(np.array([1, 0, 1]), np.array([0.9, 0.8, 0.7])))   # harus ~0.8333
print(average_precision(np.array([1, 0, 0]), np.array([0.95, 0.8, 0.9])))  # harus 1.0
print(average_precision(np.array([1, 0, 0]), np.array([0.0, 0.8, 0.9])))   # harus ~0.3333


0.8333333333333333
1.0
0.3333333333333333


### Soal 1.3 (2 poin)

Implementasikan fungsi untuk menghitung cosine similarity.

In [74]:
def cos_sim(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))


assert np.isclose(cos_sim(np.array([1, 0]), np.array([1, 0])), 1.0)
assert np.isclose(cos_sim(np.array([1, 0]), np.array([0, 1])), 0.0)
assert np.isclose(cos_sim(np.array([1, 2]), np.array([2, 4])), 1.0)

In [73]:
print(cos_sim(np.array([1, 0]), np.array([1, 0])))
print(cos_sim(np.array([1, 0]), np.array([0, 1])))
print(cos_sim(np.array([1, 2]), np.array([2, 4])))

1.0
0.0
0.9999999999999998


### Soal 1.4 (4 poin)

Gunakan metode TF-IDF untuk mengukur relevansi dari passage berdasarkan *cosine similarity* dari query dan passages. Hitung nilai *mean average precision* dari sistem yang dihasilkan. Dalam sistem ini, Anda tidak perlu membandingkan query dengan semua passages dari setiap baris di tabel. Anda hanya perlu membandingkan query dengan passages yang ada dalam satu baris.

In [78]:
from sklearn.feature_extraction.text import TfidfVectorizer

hasil_ap = []

for i in range(len(df_val)):
    query = df_val.iloc[i]['query']
    passages = df_val.iloc[i]['passages']['passage_text']
    label = df_val.iloc[i]['passages']['is_selected']

    # Gabungkan query dan passages jadi satu list teks
    semua_text = [query] + passages
    tfidf = TfidfVectorizer().fit_transform(semua_text).toarray()

    # Pastikan jumlah vektor sesuai
    if len(tfidf) == len(passages) + 1:
        query_vec = tfidf[0]
        sim = []
        for j in range(len(passages)):
            sim.append(cos_sim(query_vec, tfidf[j + 1]))

        ap = average_precision(np.array(label), np.array(sim))
        hasil_ap.append(ap)

# Hitung nilai Mean Average Precision (MAP)
print("MAP TF-IDF:", np.mean(hasil_ap))


MAP TF-IDF: nan


/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:3596: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.11/dist-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


### Soal 1.5 (2 poin)

Implementasikan sistem yang serupa seperti soal 1.3, tetapi kali ini gunakan BM25 untuk menghitung skornya. Gunakan implementasi seperti yang dicontohkan [di sini](https://pypi.org/project/rank-bm25/).

In [ ]:
from rank_bm25 import BM25Okapi

hasil_ap_bm25 = []

for i in range(3):  # Ubah ke len(df_val) untuk semua data
    query = df_val.iloc[i]['query'].lower().split()
    passages = df_val.iloc[i]['passages']['passage_text']
    label = df_val.iloc[i]['passages']['is_selected']

    token = [p.lower().split() for p in passages]
    bm25 = BM25Okapi(token)

    sim = bm25.get_scores(query)

    ap = average_precision(np.array(label), sim)
    hasil_ap_bm25.append(ap)

print("MAP BM25:", np.mean(hasil_ap_bm25))

MAP BM25: 0.15277777777777776


### Soal 1.6 (3 poin)

Bagaimana nilai MAP yang Anda dapatkan dari BM25 jika dibandingkan dengan tf-idf? Mengapa hasilnya seperti itu?

*Jawaban Anda di sini*

## 2. Dense Matrix Retrieval (10 poin)

### Soal 2.1 (2 poin)

Ambil 500 sampel (baris) dari `df_val`. Hitung nilai MAP dari sistem dengan tf-idf.

In [ ]:
np.random.seed(42)

df_sample = ... # Ganti ini dengan kode untuk mengambil sampel dari df_val

# Kode Anda di sini

### Soal 2.2 (3 poin)

Gunakan model `all-MiniLM-L6-v2` untuk menghasilkan embeddings dari query dan passages. Lalu, hitung nilai MAP dari sampel yang dihasilkan di soal 2.1.

In [ ]:
from sentence_transformers import SentenceTransformer

# Kode Anda di sini

### Soal 2.3 (3 poin)

Bagaimana nilai MAP yang Anda dapatkan dari sistem dengan embeddings jika dibandingkan dengan tf-idf? Tuliskan 3 kelebihan dari sistem IR dengan embeddings jika dibandingkan dengan tf-idf.

*Jawaban Anda di sini*

### Soal 2.4 (2 poin)

Karena dataset yang digunakan sudah menangani bagian *retrieval*, gunakan model `CrossEncoder` untuk memberikan skor dari passages yang ada. Hitung nilai MAP-nya. Apakah Anda mendapatkan nilai MAP yang lebih baik?

In [ ]:
from sentence_transformers import CrossEncoder

# Kode Anda di sini

## 3. Retrieval-Augmented Generation (25 poin)

*Acknowledgement:* Data yang digunakan pada bagian ini dikumpulkan oleh [Miguel Escobar Varela](https://miguelescobar.com/) dan tersedia secara terbuka di [HuggingFace](https://huggingface.co/datasets/mevsg/warisan-classification-data-v1).

In [ ]:
splits = {
    "train": "data/train-00000-of-00001.parquet",
    "test": "data/test-00000-of-00001.parquet",
}
df = pd.concat(
    [
        pd.read_parquet(
            "hf://datasets/mevsg/warisan-classification-data-v1/" + splits["train"]
        ),
        pd.read_parquet(
            "hf://datasets/mevsg/warisan-classification-data-v1/" + splits["test"]
        ),
    ],
    ignore_index=True
)

### Soal 3.1 (3 poin)

Dengan menggunakan model `Llama-3.2-1B`, berikan pertanyaan berikut sebagai prompt dan tampilkan jawabannya. Apa yang dapat Anda amati dari hasilnya?

In [ ]:
query = "Apa saja warisan budaya tak benda dari Jawa Barat?"

# Kode Anda di sini

*Jawaban Anda di sini*

### Soal 3.2 (2 poin)

Sekarang, Anda akan membangun sistem retrieval-augmented generation (RAG) sederhana. Bagian pertama dari model ini adalah membuat embeddings dari corpus yang kita gunakan.

Bentuk corpus dari kolom `texts`. Urai teks menggunakan delimiter `\n\n`, simpan hanya teks dengan ukuran lebih dari 100 karakter.

In [ ]:
# Kode Anda di sini
assert len(corpus) == 12037

### Soal 3.3.a (2 poin)

Buat embeddings dari corpus dengan model `all-MiniLM-L6-v2`.

In [ ]:
# Kode Anda di sini

### Soal 3.3.b (2 poin)

Buat embeddings dari corpus dengan model `intfloat/multilingual-e5-small`.

In [ ]:
# Kode Anda di sini

### Soal 3.3.c (4 poin)

Buat embedding dari variabel `query` dengan menggunakan dua model dari soal 3.3.a dan 3.3.b, lalu ambil lima dokumen dengan skor paling tinggi dari `corpus`. Anda dapat memanfaatkan fungsi *semantic search* dari Sentence Transformers.

In [ ]:
from sentence_transformers import util

# Kode Anda di sini

### Soal 3.3.d (2 poin)

Apa yang dapat Anda amati dari hasil kedua model di atas? Apa yang menyebabkan perbedaan tersebut?

*Jawaban Anda di sini*

### Soal 3.4 (3 poin)

Dengan menggunakan model `intfloat/multilingual-e5-small` untuk mencari dokumen yang relevan, rangkum hasilnya dengan model `Llama-3.2-1B`.

Catatan: Anda mungkin butuh rekayasa *prompt* agar mendapatkan hasil yang diharapkan, e.g. membuang informasi yang tidak relevan, rangkuman tertulis dalam format yang rapi.

In [ ]:
# Kode Anda di sini

### Soal 3.5 (2 poin)

Jelaskan dua contoh metrik evaluasi yang dapat digunakan untuk mengukur kualitas bagian generation dari sebuah sistem RAG.

*Jawaban Anda di sini*

### Soal 3.6 (5 poin)

Lakukan eksplorasi tambahan untuk bagian ini, misalnya:
1. Evaluasi sistem Anda dengan query yang berbeda
2. Gunakan model berbeda untuk bagian IR dari sistem
3. Gunakan model berbeda sebagai LLM
4. Gunakan OpenAI API tanpa komponen IR untuk mengevaluasi query

In [ ]:
# Kode Anda di sini